In [80]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import liana as li
import anndata
import scanpy as sc
import os 

In [81]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

# !pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
# !pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
# !pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

# Helper function for visualization.
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from utils import *

1.13.1+cu117


In [82]:
import argparse
import os.path as osp

import torch
import torch.nn.functional as F

from torch_geometric.datasets import Entities
from torch_geometric.nn import FastRGCNConv, RGCNConv, GCNConv, InnerProductDecoder, GAE, VGAE
from torch_geometric.utils import k_hop_subgraph

from torch_geometric.datasets import Planetoid
from torch_geometric.data.data import Data
from torch_geometric.transforms import NormalizeFeatures


from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import SpectralClustering
import random
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

from sklearn.metrics.pairwise import cosine_similarity

from scipy.io import mmread

# Pre-processing from original matrix


In [30]:
adata = sc.read_h5ad("/data/raw_data/Cardiac_cells/Visium-FZ_GT_P19.h5ad")

In [31]:
sc.pp.filter_genes(adata, min_cells=10)
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)
sc.pp.neighbors(adata)
sc.tl.leiden(adata)
matrix = pd.DataFrame.sparse.from_spmatrix(adata.X,index=adata.obs.index.tolist(),columns=adata.var["feature_name"].tolist())

In [32]:
matrix = pd.DataFrame.sparse.from_spmatrix(adata.X,index=adata.obs.index.tolist(),columns=adata.var["feature_name"].tolist())

In [33]:
meta = pd.DataFrame({"cell":adata.obs.index.tolist(),"labels":adata.obs["cell_type_original"].tolist()})
meta.index = meta["cell"].tolist()

In [34]:
matrix = matrix.transpose()

In [35]:
matrix = matrix[meta.index.tolist()]

In [36]:
from scipy.stats import wilcoxon
import anndata
import scanpy as sc

In [37]:
index = matrix.index.tolist()

In [38]:
meta['labels'] = meta["labels"].astype("string").tolist()

In [39]:
meta["cell"] = meta["cell"].astype("category")
meta["labels"] = meta["labels"].astype("category")

In [40]:
matrix = matrix.loc[:,~matrix.columns.duplicated()].copy()
meta = meta.loc[~meta.index.duplicated(),:].copy()
cell_group = meta.groupby("labels").count()
valid_cell_groups = cell_group[cell_group["cell"] > 1].index.tolist()
meta = meta[meta["labels"].isin(valid_cell_groups)]
meta["labels"] = meta["labels"].cat.remove_unused_categories()
matrix = matrix[meta["cell"].tolist()]



In [41]:
adata = anndata.AnnData(matrix.transpose())

In [42]:
sc.pp.filter_genes(adata, min_cells=10)


In [43]:
#adata = anndata.AnnData(matrix.transpose())
adata.obs["labels"] =  meta["labels"].astype("category")

In [44]:
sc.pp.normalize_total(adata)

In [45]:
adata.obs = meta

In [46]:
#sc.tl.rank_genes_groups(adata, 'labels')
cell_groups = meta['labels'].unique().tolist()
matrix_list = {}
for i in cell_groups:
    cells = meta[meta["labels"]==i].index.tolist()
    temp_matrix = matrix[cells]
    matrix_list[i] = (temp_matrix.mean(axis=1)[temp_matrix.mean(axis=1) > 0].index.tolist())

In [47]:
adata = adata.transpose()

In [48]:
#adata.obs["Gene"] = matrix.index.astype("category")

In [49]:
# pval_df = pd.DataFrame.from_records(adata.uns["rank_genes_groups"]["pvals"])
# pval_df.index = adata.obs.index.tolist()

In [50]:
cell_type_df = matrix_list

In [51]:
# cell_type_df = {}
# for i in pval_df.columns.tolist():
#     sub = pval_df[i]
#     sub = sub[sub < 0.05]
#     cell_type_df[i] = sub.index.tolist()

In [52]:
#cell_type_df = matrix_list

In [85]:
#mean_expression = np.mean(matrix.values[matrix.values != 0])
nodes = pd.DataFrame({"category":[],"identifier":[]})

In [86]:
LR_nodes = pd.read_csv("/data/LR_database/OmniPath_nodes.csv",index_col=0)
Omnipath_network = pd.read_csv("/data/LR_database/OmniPath_interactions.csv",index_col=0)

In [87]:
ligands = LR_nodes[LR_nodes["category"]=="Ligand"]["identifier"].tolist()
receptors = LR_nodes[LR_nodes["category"]=="Receptor"]["identifier"].tolist()

In [88]:
ligand_list = []
receptor_list = []
new_cell_df = {}
for i in cell_type_df.keys():
    ligand_list.extend(list(set(ligands) & set(cell_type_df[i])))
    receptor_list.extend(list(set(receptors) & set(cell_type_df[i])))
    new_cell_df[i] = [list(set(ligands) & set(cell_type_df[i])),list(set(receptors) & set(cell_type_df[i]))]

In [89]:
for i in new_cell_df.keys():
    new_cell_df[i][0] = [j+"_Ligand" for j in new_cell_df[i][0]]
    new_cell_df[i][1] = [j+"_Receptor" for j in new_cell_df[i][1]]

In [90]:
ligand_list = list(set(ligand_list))
receptor_list = list(set(receptor_list))

In [91]:
ligand_list = [i+"_Ligand" for i in ligand_list]
receptor_list = [i+"_Receptor" for i in receptor_list]

In [92]:
nodes = pd.concat([nodes,pd.DataFrame({"category":["Cell Group"] * len(list(cell_type_df.keys())), "identifier":list(cell_type_df.keys())})])
nodes = pd.concat([nodes,pd.DataFrame({"category":["Ligand"] * len(ligand_list), "identifier":ligand_list})])
nodes = pd.concat([nodes,pd.DataFrame({"category":["Receptor"] * len(receptor_list), "identifier":receptor_list})])

In [93]:
new_identifier = [row["identifier"] + "_" + row["category"] for index,row in LR_nodes.iterrows()]

In [94]:
LR_nodes["identifier"] = new_identifier

In [95]:
nodes["Id"] = range(0,nodes.shape[0])
nodes = nodes[["Id","category","identifier"]]

In [96]:
nodes.index = nodes.index.astype('int')
nodes["Id"] = nodes["Id"].astype('int')

In [97]:
meta.index = meta["cell"]

In [98]:
interactions = pd.DataFrame({"Src":[],"Dst":[],"Weight":[],"edge_type":[]})

In [99]:
LR_nodes.index = LR_nodes["Id"].tolist()

In [100]:
Omnipath_network["Src"] = LR_nodes.loc[Omnipath_network["Src"].tolist()]["identifier"].tolist()
Omnipath_network["Dst"] = LR_nodes.loc[Omnipath_network["Dst"].tolist()]["identifier"].tolist()


In [101]:
source_list = []
dest_list = []
weight_list = []
edge_type_list = []
for i in new_cell_df.keys():
    source_list.extend([i] * (len(new_cell_df[i][0]) + len(new_cell_df[i][1])))
    dest_list.extend(new_cell_df[i][0])
    dest_list.extend(new_cell_df[i][1])
    weight_list.extend([1] * (len(new_cell_df[i][0]) + len(new_cell_df[i][1])))
    edge_type_list.extend([1] * (len(new_cell_df[i][0]) + len(new_cell_df[i][1])))

In [102]:
interactions["Src"] = source_list
interactions["Dst"] = dest_list
interactions["Weight"] = weight_list
interactions["edge_type"] = edge_type_list

In [103]:
nodes.index = nodes["identifier"].tolist()

In [104]:
nodes = nodes.drop_duplicates("identifier")
nodes["Id"] = range(0,nodes.shape[0])

In [105]:
interactions["Src"] = nodes.loc[interactions["Src"].tolist()]["Id"].tolist()
interactions["Dst"] = nodes.loc[interactions["Dst"].tolist()]["Id"].tolist()

In [106]:
first_data,first_nodes,first_interactions = make_dataset(nodes,interactions,first=False,pathway_encode=False)

# Link Prediction with GAT

In [107]:
from torch_geometric.nn import GATConv
class GAT(torch.nn.Module):
    def __init__(self,data,num_classes=3):
        super(GAT, self).__init__()
        self.hid = 3
        self.in_head = 3
        self.out_head = 3
        
        
        self.conv1 = GATConv(data.x.shape[1], self.hid, heads=self.in_head, dropout=0.6)
        self.conv2 = GATConv(self.hid*self.in_head,num_classes, concat=False,
                             heads=self.out_head, dropout=0.6)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
                
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.elu(x)
        x = F.logsigmoid(x)
        return x
    
    
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = "cpu"
    

# Get Omnipath embedding

In [108]:
LR_nodes = LR_nodes[(LR_nodes["identifier"].isin(ligand_list)) | (LR_nodes["identifier"].isin(receptor_list))]

In [109]:
Omnipath_network = Omnipath_network[(Omnipath_network["Src"].isin(LR_nodes["identifier"].tolist())) & (Omnipath_network["Dst"].isin(LR_nodes["identifier"].tolist()))]

In [110]:
LR_nodes["Id"] = range(0,LR_nodes.shape[0])

In [111]:
LR_nodes.index = LR_nodes["identifier"].tolist()
Omnipath_network["Src"] = LR_nodes.loc[Omnipath_network["Src"].tolist()]["Id"].tolist()
Omnipath_network["Dst"] = LR_nodes.loc[Omnipath_network["Dst"].tolist()]["Id"].tolist()

In [112]:
LR_nodes = LR_nodes[(LR_nodes["Id"].isin(Omnipath_network["Src"].tolist())) | (LR_nodes["Id"].isin(Omnipath_network["Dst"].tolist()))]

In [113]:
LR_nodes.index = LR_nodes["Id"].tolist()
Omnipath_network["Src"] = LR_nodes.loc[Omnipath_network["Src"].tolist()]["identifier"].tolist()
Omnipath_network["Dst"] = LR_nodes.loc[Omnipath_network["Dst"].tolist()]["identifier"].tolist()

In [114]:
LR_nodes.index = LR_nodes["identifier"].tolist()
LR_nodes["Id"] = range(0,LR_nodes.shape[0])
Omnipath_network["Src"] = LR_nodes.loc[Omnipath_network["Src"].tolist()]["Id"].tolist()
Omnipath_network["Dst"] = LR_nodes.loc[Omnipath_network["Dst"].tolist()]["Id"].tolist()

In [115]:
Omnipath_data,Omnipath_nodes,Omnipath_interactions = make_dataset(LR_nodes,Omnipath_network,first=False,pathway_encode=False)

In [116]:
node_info = pd.DataFrame(np.zeros((Omnipath_nodes.shape[0],Omnipath_nodes.shape[0])),index=Omnipath_nodes["identifier"].tolist(),columns=Omnipath_nodes["identifier"].tolist())

In [117]:
temp_identifiers = [i.split("_")[0] for i in Omnipath_nodes["identifier"].tolist()]

In [120]:
complexes = pd.read_csv("/data/LR_database/complexes.csv")
complexes = complexes[complexes["member"].isin(temp_identifiers)]

In [121]:
temp_nodes = Omnipath_nodes.copy()
temp_nodes.index = temp_identifiers
temp_nodes = temp_nodes[~temp_nodes.index.duplicated(keep='first')]

In [122]:
complexes["member"] = temp_nodes.loc[complexes["member"].tolist()]["identifier"].tolist()

In [123]:
group_complex = complexes.groupby("complex").agg(list)

In [124]:
group_complex.index=range(0,group_complex.shape[0])

In [125]:
for index,row in group_complex.iterrows():
    node_info.loc[list(set(row["member"])),list(set(row["member"]))] = index

In [126]:
# for i in group_complex["member"].tolist():
#     node_info.loc[list(set(i)),list(set(i))] = 1

In [128]:
pathways = pd.read_csv("/data/LR_database/kegg_pathways.csv",index_col=0)
pathways = pathways[pathways["genesymbol"].isin(temp_identifiers)]
pathways["genesymbol"] = temp_nodes.loc[pathways["genesymbol"].tolist()]["identifier"].tolist()
group_pathway = pathways.groupby("pathway").agg(list)

In [129]:
group_pathway.index=range(0,group_pathway.shape[0])

In [130]:
for index,row in group_pathway.iterrows():
    node_info.loc[list(set(row["genesymbol"])),list(set(row["genesymbol"]))] += index

In [132]:
truth_info = pd.DataFrame(np.zeros((Omnipath_nodes.shape[0],Omnipath_nodes.shape[0])),index=Omnipath_nodes["identifier"].tolist(),columns=Omnipath_nodes["identifier"].tolist())

In [133]:
Omnipath_nodes.index = Omnipath_nodes["Id"].tolist()

In [134]:
ident_interactions = Omnipath_interactions.copy()
ident_interactions["Src"] = Omnipath_nodes.loc[ident_interactions["Src"].tolist()]["identifier"].tolist()
ident_interactions["Dst"] = Omnipath_nodes.loc[ident_interactions["Dst"].tolist()]["identifier"].tolist()

In [135]:
for index,row in ident_interactions.iterrows():
    truth_info.loc[row["Src"],row["Dst"]] = 1

In [136]:
ligands = Omnipath_nodes[Omnipath_nodes["category"]=="Ligand"]["identifier"].tolist()
receptors = Omnipath_nodes[Omnipath_nodes["category"]=="Receptor"]["identifier"].tolist()
#truth_info = truth_info.loc[ligands,receptors]
truth_info = torch.Tensor(truth_info.values).to(device)


In [137]:
ligands = Omnipath_nodes[Omnipath_nodes["category"]=="Ligand"]["Id"].tolist()
receptors = Omnipath_nodes[Omnipath_nodes["category"]=="Receptor"]["Id"].tolist()


In [138]:
ident_interactions = ident_interactions.drop_duplicates("Src")
ident_interactions = ident_interactions.drop_duplicates("Dst")

In [139]:
ident_interactions.index = range(0,ident_interactions.shape[0])

In [140]:
truth_list = []
for i in Omnipath_nodes["identifier"].tolist():
    if "Ligand" in i:
        if i in ident_interactions["Src"].tolist():
            #truth_list.append(ident_interactions[ident_interactions["Src"]==i].index.tolist()[0] + 1)
            truth_list.append(1)
        else:
            truth_list.append(0)
    if "Receptor" in i:
        if i in ident_interactions["Dst"].tolist():
            #truth_list.append(ident_interactions[ident_interactions["Dst"]==i].index.tolist()[0] + 1)
            truth_list.append(1)
        else:
            truth_list.append(0)

In [141]:
node_info.values[np.where(np.isnan(node_info.values))] = 0
node_info.values[np.where(np.isinf(node_info.values))] = 0

In [142]:
Omnipath_data.x = torch.Tensor(node_info.values)

In [143]:
#Omnipath_data.y = torch.Tensor(truth_list).type(torch.LongTensor)
Omnipath_data.y = truth_info

In [144]:
Omnipath_nodes.index = Omnipath_nodes["Id"].tolist()

In [145]:
Omnipath_interactions["Src"] = [Omnipath_nodes.loc[i]["identifier"] for i in Omnipath_interactions["Src"].tolist()]
Omnipath_interactions["Dst"] = [Omnipath_nodes.loc[i]["identifier"] for i in Omnipath_interactions["Dst"].tolist()]

In [146]:
#edge_weights = [max(node_info.loc[i,j],node_info.loc[j,i]) for i,j in zip(Omnipath_interactions["Src"].tolist(),Omnipath_interactions["Dst"].tolist())]
edge_weights = [1 for i,j in zip(Omnipath_interactions["Src"].tolist(),Omnipath_interactions["Dst"].tolist())]

# Random Walks

In [147]:
from torch_geometric.nn import Node2Vec
import os.path as osp
import torch
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from torch_geometric.datasets import Planetoid
from tqdm.notebook import tqdm

In [148]:
data = Omnipath_data

## data loader

In [149]:
model = Node2Vec(data.edge_index, embedding_dim=2, walk_length=40,
                 context_size=40, walks_per_node=10,
                 num_negative_samples=1, p=1, q=1, sparse=True).to(device)

loader = model.loader(batch_size=2, shuffle=True, num_workers=8)
optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.01)

In [150]:
def train():
    model.train()
    total_loss = 0
    for pos_rw, neg_rw in (loader):
        optimizer.zero_grad()
        loss = model.loss(pos_rw.to(device), neg_rw.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

In [151]:
@torch.no_grad()
def test():
    model.eval()
    z = model()
    acc = model.test(z[data.train_mask], data.y[data.train_mask],
                     z[data.test_mask], data.y[data.test_mask],
                     max_iter=150)
    return acc

In [153]:
model = torch.load("/data/models/Cardiac_cells/FZ_GT_P19/Omnipath.pt")

In [154]:
z = model(torch.arange(data.num_nodes, device=device)).detach()

In [155]:
ligand_ids = Omnipath_nodes[Omnipath_nodes["category"].str.contains("Ligand")]["Id"].tolist()
receptor_ids = Omnipath_nodes[Omnipath_nodes["category"].str.contains("Receptor")]["Id"].tolist()

In [156]:
ligand_embeddings = z[ligand_ids,:]
receptor_embeddings = z[receptor_ids,:]

In [157]:
total_embeddings = torch.inner(ligand_embeddings,receptor_embeddings)

In [158]:
total_embeddings_df = pd.DataFrame(total_embeddings.numpy(),index=Omnipath_nodes[Omnipath_nodes["category"].str.contains("Ligand")]["identifier"].tolist(),columns=Omnipath_nodes[Omnipath_nodes["category"].str.contains("Receptor")]["identifier"].tolist())

# make cell -> ligand and receptor graphs 

In [159]:
cell_LR_data,cell_LR_nodes,cell_LR_ints = make_dataset(nodes,interactions,first=False,pathway_encode=False)

In [160]:
full_matrix = pd.DataFrame(np.zeros((cell_LR_nodes.shape[0],cell_LR_nodes.shape[0])),index=cell_LR_nodes["identifier"].tolist(),columns=cell_LR_nodes["identifier"].tolist())

In [161]:
ligands = Omnipath_nodes[Omnipath_nodes["category"]=="Ligand"]["Id"].tolist()
receptors = Omnipath_nodes[Omnipath_nodes["category"]=="Receptor"]["Id"].tolist()

In [162]:
total_out_df = total_embeddings_df

In [163]:
#matrix = matrix.transpose()

In [164]:
gene_mean = matrix.mean(axis=1)

In [166]:
Omnipath_nodes.index = [i.split("_")[0] for i in Omnipath_nodes["identifier"].tolist()]

In [167]:
Omnipath_nodes = Omnipath_nodes.loc[~Omnipath_nodes.index.duplicated(),:].copy()


In [168]:
gene_mean = gene_mean.loc[Omnipath_nodes.index.tolist()]

In [169]:
gene_mean.index = Omnipath_nodes["identifier"].tolist()

In [170]:
ligands = [i for i in gene_mean.index.tolist() if "Ligand" in i]
receptors = [i for i in gene_mean.index.tolist() if "Receptor" in i]
ligands = list(set(full_matrix.index.tolist()) & set(ligands))
receptors = list(set(full_matrix.index.tolist()) & set(receptors))

In [171]:
for i,j in zip(ligands,receptors):
    if (i in gene_mean.index.tolist()) and (j in gene_mean.index.tolist()):
        full_matrix.loc[i,j] = gene_mean.loc[i]*gene_mean.loc[j]

In [172]:
ligands = list(set(ligands) & set(total_out_df.index.tolist()))
receptors = list(set(receptors) & set(total_out_df.columns.tolist()))

In [173]:
for i,j in zip(ligands,receptors):
    full_matrix.loc[i,j] += total_out_df.loc[i,j]
    

In [174]:
#full_matrix.loc[true_df.index.tolist(),true_df.columns.tolist()] = true_df.values

In [175]:
cell_groups = meta["labels"].unique().tolist()

In [176]:
ligands = cell_LR_nodes[cell_LR_nodes["category"]=="Ligand"]["identifier"].tolist()
ligands = [i.split("_")[0] for i in ligands]
ligand_matrix = matrix.loc[ligands]

In [177]:
mean_dict = {}
for i in cell_groups:
    cells = meta[meta["labels"]==i]["cell"].tolist()
    mean_dict[i] = ligand_matrix[cells].mean(axis=1)

In [178]:
for i in mean_dict.keys():
    temp_index = [i+"_Ligand" for i in mean_dict[i].index.tolist()]
    full_matrix.loc[i,temp_index] = mean_dict[i].values
    full_matrix.loc[temp_index,i] = mean_dict[i].values

In [179]:
receptors = cell_LR_nodes[cell_LR_nodes["category"]=="Receptor"]["identifier"].tolist()
receptors = [i.split("_")[0] for i in receptors]
receptors_matrix = matrix.loc[receptors]

In [180]:
mean_dict = {}
for i in cell_groups:
    cells = meta[meta["labels"]==i]["cell"].tolist()
    mean_dict[i] = receptors_matrix[cells].mean(axis=1)

In [181]:
for i in mean_dict.keys():
    temp_index = [i+"_Receptor" for i in mean_dict[i].index.tolist()]
    full_matrix.loc[i,temp_index] = mean_dict[i].values
    full_matrix.loc[temp_index,i] = mean_dict[i].values

In [182]:
#full_matrix = (full_matrix - np.min(full_matrix)) / (np.max(full_matrix) - np.min(full_matrix))

In [183]:
full_matrix.values[np.where(np.isnan(full_matrix.values))] = 0
full_matrix.values[np.where(np.isinf(full_matrix.values))] = 0

In [184]:
adata = sc.read_h5ad("/data/raw_data/Cardiac_cells/Visium-FZ_GT_P19.h5ad")
adata = adata[meta.index.tolist()]

In [191]:
spatial_coordinates = adata.obs[['Adipocyte', 'Cardiomyocyte','Endothelial', 'Fibroblast', 'Lymphoid', 'Mast', 'Myeloid', 'Neuronal','Pericyte']]

In [192]:
cell_groups = meta["labels"].unique().tolist()

In [195]:
j = "Pericyte"

In [196]:
 cells = meta[meta["labels"]==i].index.tolist()

In [207]:
spatial_coords = spatial_coordinates.loc[cells].values

In [202]:
import math

In [281]:
LRs = list(set(nodes[nodes["category"]=="Ligand"]["identifier"].tolist() + nodes[nodes["category"]=="Receptor"]["identifier"].tolist()))

In [284]:
new_matrix = matrix.loc[[i.split("_")[0] for i in LRs]]

In [285]:
new_matrix = matrix.transpose()

In [219]:
#deconvolution embedding
spatial_dict = {}
for i in cell_groups:
    cells = meta[meta["labels"]==i].index.tolist()
    spatial_coords = [tuple(i) for i in spatial_coordinates.loc[cells].values]
    for j in cell_groups:
        if j != i:
            second_cells = meta[meta["labels"]==j].index.tolist()
            second_spatial_coords = [tuple(i) for i in spatial_coordinates.loc[second_cells].values]
            min_list = []
            for k in spatial_coords:
                min_list.append(min([math.dist(k,l) for l in second_spatial_coords]))
            spatial_dict[(i,j)] = min(min_list)

            

In [291]:
import numpy as np

In [296]:
    spatial_coords = new_matrix.loc[cells].values


In [300]:
np.mean(spatial_coords)

0.09168209836511541

In [305]:
new_matrix.loc[cells].mean(axis=0).values

array([0.0354877 , 0.19403787, 0.14786798, ..., 0.10677629, 0.03728484,
       0.06707481])

In [309]:
cell_groups

['Fibroblast',
 'Endothelial',
 'Myeloid',
 'Cardiomyocyte',
 'vSMCs',
 'Mast',
 'Pericyte',
 'Neuronal']

In [311]:
#expression embedding
spatial_dict = {}
for i in cell_groups:
    print(i)
    cells = meta[meta["labels"]==i].index.tolist()
    spatial_coords = new_matrix.loc[cells].mean(axis=0).values
    for j in cell_groups:
        if j != i:
            second_cells = meta[meta["labels"]==j].index.tolist()
            second_spatial_coords = new_matrix.loc[second_cells].mean(axis=0).values
            min_list = []
            min_list.append(math.dist(spatial_coords,second_spatial_coords))
            spatial_dict[(i,j)] = min(min_list)


Fibroblast
Endothelial
Myeloid
Cardiomyocyte
vSMCs
Mast
Pericyte
Neuronal


In [312]:
for k in spatial_dict.keys():
    full_matrix.loc[k[0],k[1]] = spatial_dict[k]
    full_matrix.loc[k[1],k[0]] = spatial_dict[k]


In [313]:
cell_LR_data.x = torch.Tensor(full_matrix.values)

In [314]:
LR_ids = cell_LR_nodes[(cell_LR_nodes["category"]=="Ligand") | (cell_LR_nodes["category"]=="Receptor")]["Id"].tolist()

In [315]:
# true_values["Src"] = [i + "_Ligand" for i in true_values["Src"].tolist()]
# true_values["Dst"] = [i + "_Receptor" for i in true_values["Dst"].tolist()]

In [316]:
cell_groups = cell_LR_nodes[cell_LR_nodes["category"]=="Cell Group"]['identifier'].tolist()

In [317]:
truth_list = []
for i in cell_LR_nodes["identifier"].tolist():
    if "Ligand" in i:
        if i in ident_interactions["Src"].tolist():
            #truth_list.append(ident_interactions[ident_interactions["Src"]==i].index.tolist()[0] + 1)
            truth_list.append(1)
        else:
            truth_list.append(0)
    elif "Receptor" in i:
        if i in ident_interactions["Dst"].tolist():
            #truth_list.append(ident_interactions[ident_interactions["Dst"]==i].index.tolist()[0] + 1)
            truth_list.append(1)
        else:
            truth_list.append(0)
    else:
        truth_list.append(2)

In [318]:
cell_LR_data.y = torch.Tensor(truth_list).type(torch.LongTensor)

In [319]:
truth_array = np.array(truth_list)
positive_classes = np.where(truth_array==1)[0].tolist()
negative_classes = np.where(truth_array==0)[0].tolist()[:len(positive_classes)]

In [320]:
new_train_mask = np.array([False]*truth_array.shape[0])
new_train_mask[positive_classes + negative_classes] = True


In [321]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = "cpu"

model = GAT(cell_LR_data,num_classes=2).to(device)
data = cell_LR_data.to(device)

In [322]:
data.train_mask = torch.Tensor(new_train_mask).type(torch.LongTensor)

In [323]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

In [324]:
ligands = cell_LR_nodes[cell_LR_nodes["category"]=="Ligand"]["Id"].tolist()
receptors = cell_LR_nodes[cell_LR_nodes["category"]=="Receptor"]["Id"].tolist()

In [325]:
truth_df = full_matrix.loc[cell_LR_nodes[cell_LR_nodes["category"]=="Ligand"]["identifier"].tolist(),cell_LR_nodes[cell_LR_nodes["category"]=="Receptor"]["identifier"].tolist()]

In [326]:
truth_Tensor = torch.Tensor(truth_df.values).to(device)

In [327]:
for epoch in range(100):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[new_train_mask],data.y[new_train_mask])
    ligand_out = out[ligands,:]
    receptor_out = out[receptors,:]
    total_out = torch.inner(ligand_out,receptor_out)
    #loss = criterion(out[LR_ids],data.y)
    #loss = criterion(total_out,truth_Tensor)
    #loss = criterion(out[new_train_mask],data.y[new_train_mask])
    if epoch%20 == 0:
        print(loss)
    
    loss.backward()
    optimizer.step()
    

tensor(0.8972, grad_fn=<NllLossBackward0>)
tensor(0.2932, grad_fn=<NllLossBackward0>)
tensor(0.2099, grad_fn=<NllLossBackward0>)
tensor(0.3331, grad_fn=<NllLossBackward0>)
tensor(0.3276, grad_fn=<NllLossBackward0>)


In [328]:
model.eval()
cell_LR_out = model(data)

In [329]:
out = cell_LR_out

In [330]:
ligands = cell_LR_nodes[cell_LR_nodes["category"]=="Ligand"]["Id"].tolist()
receptors = cell_LR_nodes[cell_LR_nodes["category"]=="Receptor"]["Id"].tolist()

In [331]:
ligand_out = cell_LR_out[ligands,:]
receptor_out = cell_LR_out[receptors,:]
_,ligand_pred = ligand_out.max(dim=1)
_,receptor_pred = receptor_out.max(dim=1)

total_out = torch.inner(ligand_out,receptor_out).cpu().detach().numpy()

In [332]:
cell_LR_nodes.index = cell_LR_nodes["Id"].tolist()


In [333]:
ligand_df = cell_LR_nodes[cell_LR_nodes["category"]=="Ligand"]
receptor_df = cell_LR_nodes[cell_LR_nodes["category"]=="Receptor"]
ligand_df.index = range(ligand_df.shape[0])
receptor_df.index = range(receptor_df.shape[0])

In [334]:
valid_ligands = ligand_df
valid_receptors = receptor_df


In [335]:
ligand_pred = ligand_pred.cpu().detach().numpy()
receptor_pred = receptor_pred.cpu().detach().numpy()

In [336]:
from sklearn.metrics import recall_score, precision_score

In [337]:
precision_score(truth_array[ligands], ligand_pred, average='binary')

0.3526530612244898

In [338]:
precision_score(truth_array[receptors], receptor_pred, average='binary')

0.4267369114257491

In [339]:
recall = recall_score(truth_array[ligands], ligand_pred, average='binary')
print('Recall: %.3f' % recall)

Recall: 1.000


In [340]:
recall = recall_score(truth_array[receptors], receptor_pred, average='binary')
print('Recall: %.3f' % recall)

Recall: 1.000


In [341]:
ligand_out = ligand_out[valid_ligands.index.tolist()]
receptor_out = receptor_out[valid_receptors.index.tolist()]

In [342]:
ligand_nodes = cell_LR_nodes[cell_LR_nodes["category"] == "Ligand"]
ligand_nodes.index = range(0,ligand_nodes.shape[0])
ligand_idents = ligand_nodes.iloc[np.where(ligand_pred==1)]['identifier'].tolist()

In [343]:
total_out_df = pd.DataFrame(total_out,index=valid_ligands["identifier"].tolist(),columns=valid_receptors["identifier"].tolist())

In [344]:
# true_values["Prob"] = (true_values["Prob"] - np.min(true_values["Prob"])) / (np.max(true_values["Prob"]) - np.min(true_values["Prob"]))

In [345]:
# true_values = true_values.drop_duplicates()

In [346]:
# link_df["Prob"] = (link_df["Prob"] - np.min(link_df["Prob"])) / (np.max(link_df["Prob"]) - np.min(link_df["Prob"]))
# #link_df["Src"] = [i+"_Ligand" for i in link_df["Src"].tolist()]
# #link_df["Dst"] = [i+"_Receptor" for i in link_df["Dst"].tolist()]

In [347]:
# for index,row in true_values.iterrows():
#     total_out_df.loc[row["Src"],row["Dst"]] += row["Prob"]

In [348]:
indicies = np.where(total_out_df.values > 0)
source = list(indicies[0])
dest = list(indicies[1])

In [349]:
index_df = pd.DataFrame({"Id":range(0,total_out_df.shape[0]),"identifier":total_out_df.index.tolist()})
column_df = pd.DataFrame({"Id":range(0,total_out_df.shape[1]),"identifier":total_out_df.columns.tolist()})

In [350]:
source_list = index_df.loc[source]["identifier"].tolist()
dest_list = column_df.loc[dest]["identifier"].tolist()

In [351]:
total_link_df = pd.DataFrame({"Src":source_list,"Dst":dest_list,"Prob":total_out_df.values[indicies]})

In [352]:
total_link_df = total_link_df.sort_values("Prob",ascending=False)

In [353]:
Omnipath_db = pd.read_csv("/data/LR_database/Omnipath_database.csv")

In [354]:
total_link_df["Src"] = [i.split("_")[0] for i in total_link_df["Src"].tolist()]
total_link_df["Dst"] = [i.split("_")[0] for i in total_link_df["Dst"].tolist()]

In [355]:
total_link_df = total_link_df.drop_duplicates()

In [356]:
Omnipath_db = Omnipath_db.drop_duplicates(["from","to"])

In [357]:
pd.merge(total_link_df.head(10000),Omnipath_db,left_on=["Src","Dst"],right_on=["from","to"])

,Src,Dst,Prob,Unnamed: 0,from,to,sources
0,PTPN22,ZAP70,0.001870,20270,PTPN22,ZAP70,DEPOD;HPRD;IntAct;KEA;Kinexus_KEA;ProtMapper;R...
1,PTPN22,LCK,0.001870,34133,PTPN22,LCK,DEPOD;HPRD;IntAct;KEA;Kinexus_KEA;ProtMapper;R...
2,LCK,CD3G,0.001870,15192,LCK,CD3G,SIGNOR;Wang
3,MATK,LCK,0.001830,85846,MATK,LCK,iPTMnet
4,ZAP70,LCK,0.001830,2814,ZAP70,LCK,BioGRID;Cui2007;DOMINO;HPRD;InnateDB;IntAct;Li...
5,LCK,ZAP70,0.001830,2813,LCK,ZAP70,BioGRID;Cui2007;DOMINO;HPRD;HPRD-phos;HPRD_KEA...
6,ELANE,SLPI,0.001830,88996,ELANE,SLPI,HPRD;Wang
7,SLPI,ELANE,0.001830,88995,SLPI,ELANE,HPRD;Wang
8,NTRK1,SHC3,0.001830,78311,NTRK1,SHC3,HPRD;MIMP;PhosphoPoint;PhosphoSite;PhosphoSite...
9,NTRK1,MATK,0.001830,7655,NTRK1,MATK,HPRD;Lit-BM-17;PhosphoPoint;SignaLink3;Wang


In [358]:
LR_out = cell_LR_out[valid_ligands["Id"].tolist() + valid_receptors["Id"].tolist(),:]
cell_group_out = cell_LR_out[cell_LR_nodes[cell_LR_nodes["category"]=="Cell Group"]["Id"].tolist(),:]
cell_LR_out = torch.inner(LR_out,cell_group_out).cpu().detach().numpy()

In [359]:
cell_LR_df = pd.DataFrame(cell_LR_out,index=valid_ligands["identifier"].tolist() + valid_receptors["identifier"].tolist(),columns=cell_LR_nodes[cell_LR_nodes["category"]=="Cell Group"]["identifier"].tolist())

In [360]:
ligands = nodes[nodes["category"]=="Ligand"]["identifier"].tolist()
receptors = nodes[nodes["category"]=="Receptor"]["identifier"].tolist()
expression_df = matrix

In [361]:
cell_groups = meta["labels"].unique().tolist()


# In[30]:


mean_matrix = pd.DataFrame(columns=cell_groups,index=ligands+receptors)


# In[31]:


for i in cell_groups:
    cells = meta[meta["labels"]==i]["cell"].tolist()
    temp_ligands = [i.split("_")[0] for i in ligands]
    ligand_df = expression_df[cells].mean(axis=1).loc[temp_ligands]
    ligand_df.index = [i+"_Ligand" for i in ligand_df.index.tolist()]
    temp_receptors = receptors = [i.split("_")[0] for i in receptors]
    receptor_df = expression_df[cells].mean(axis=1).loc[temp_receptors]
    receptor_df.index = [i+"_Receptor" for i in receptor_df.index.tolist()]
    total_df = pd.concat([ligand_df,receptor_df])
    mean_matrix[i] = total_df.tolist()

In [362]:
ligands = cell_LR_nodes[cell_LR_nodes["category"]=="Ligand"]["Id"].tolist()
receptors = cell_LR_nodes[cell_LR_nodes["category"]=="Receptor"]["Id"].tolist()

In [363]:
interacting_ligands = list(set(ligands) & set(cell_LR_ints["Dst"].tolist()))
interacting_receptors = list(set(receptors) & set(cell_LR_ints["Dst"].tolist()))


# In[99]:


cell_LR_ints.index = cell_LR_ints["Dst"].tolist()
ligand_cells = cell_LR_ints.loc[interacting_ligands]["Src"].unique().tolist()
receptor_cells = cell_LR_ints.loc[interacting_receptors]["Src"].unique().tolist()

In [364]:
cell_LR_out = torch.Tensor(cell_LR_out)

In [365]:
ligand_cell_out = out[ligand_cells,:]
ligand_out = out[ligands,:]
total_ligand_out = torch.inner(ligand_out,ligand_cell_out).cpu().detach().numpy()
receptor_cell_out = out[receptor_cells,:]
receptor_out = out[receptors,:]
total_receptor_out = torch.inner(receptor_out,receptor_cell_out).cpu().detach().numpy()

In [366]:
ligand_matrix = mean_matrix[mean_matrix.index.str.contains("Ligand")]
receptor_matrix = mean_matrix[mean_matrix.index.str.contains("Receptor")]


# In[112]:


ligand_cell_out = np.multiply(ligand_matrix,total_ligand_out)
receptor_cell_out = np.multiply(receptor_matrix,total_receptor_out)


# In[113]:


ligand_maxes = (ligand_cell_out.idxmax(axis=1))
receptor_maxes = (receptor_cell_out.idxmax(axis=1))


In [367]:
ligand_maxes.index = [i.split("_")[0] for i in ligand_maxes.index.tolist()]
receptor_maxes.index = [i.split("_")[0] for i in receptor_maxes.index.tolist()]

In [368]:
total_link_df["Src Cell"] = ligand_maxes.loc[total_link_df['Src']].tolist()
total_link_df["Dst Cell"] = receptor_maxes.loc[total_link_df['Dst']].tolist()


In [369]:
total_link_df.to_csv("/results/gene_spatial.csv")